In [1]:
from pandas.core.frame import DataFrame
import pandas
import pandas as pd
import pandas_ta as ta

import random
import pandas as pd

from deap import gp, creator, base, tools

class bool_series:
    pass

class signals:
    pass

class comparable_series:
    pass

class percent_series:
    pass

pset = gp.PrimitiveSetTyped("MAIN", [DataFrame], signals)

pset.addEphemeralConstant("rand_float", lambda: random.random(), float)

def series_concate(u, v):
    return pd.concat([u, v], axis = 1)
pset.addPrimitive(series_concate, [bool_series, bool_series], signals)

def series_and(u, v):
    return u & v
pset.addPrimitive(series_and, [bool_series, bool_series], bool_series)

def series_or(u, v):
    return u | v
pset.addPrimitive(series_or, [bool_series, bool_series], bool_series)

def series_not(u):
    return ~u
pset.addPrimitive(series_not, [bool_series], bool_series)

def series_lt(u, v):
    return u.le(v)
pset.addPrimitive(series_lt, [comparable_series, comparable_series], bool_series)

def series_lt_float(u, v):
    return u < v
pset.addPrimitive(series_lt_float, [percent_series, float], bool_series)

def series_gt_float(u, v):
    return u > v
pset.addPrimitive(series_gt_float, [percent_series, float], bool_series)

##################################################################################################

def price(u):
    return u['close']
pset.addPrimitive(price, [DataFrame], comparable_series)

def percent_diff(u, v):
    if u.eq(v['close']):
        return u.pct_change()
    return v['close'].substract(u).divide(v['close'])
pset.addPrimitive(percent_diff, [comparable_series, DataFrame], percent_series)

class ema_length:
    pass
class ema_offset:
    pass
pset.addEphemeralConstant("ema_length", lambda: random.randint(1, 250), ema_length)
pset.addEphemeralConstant("ema_offset", lambda: random.randint(0, 25), ema_offset)
def ema(df, length, offset):
    return df.ta.ema(length = length, offset = offset)
pset.addPrimitive(ema, [DataFrame, ema_length, ema_offset], comparable_series)

class rsi_length:
    pass
class rsi_drift:
    pass
class rsi_offset:
    pass
pset.addEphemeralConstant("rsi_length", lambda: random.randint(1, 250), rsi_length)
pset.addEphemeralConstant("rsi_drift", lambda: random.randint(0, 25), rsi_drift)
pset.addEphemeralConstant("rsi_offset", lambda: random.randint(0, 25), rsi_offset)
def rsi(df, length, drift, offset):
    return df.ta.rsi(length = length, drift = drift, offset = offset)/100
pset.addPrimitive(rsi, [DataFrame, rsi_length, rsi_drift, rsi_offset], percent_series)

pset.renameArguments(ARG0="df")

In [2]:
import sys, inspect
def generate_safe(pset, min_, max_, terminal_types, type_=None):
    if type_ is None:
        type_ = pset.ret
    expr = []
    height = random.randint(min_, max_)
    stack = [(0, type_)]
    while len(stack) != 0:
        depth, type_ = stack.pop()
        if type_ in terminal_types:
            try:
                term = random.choice(pset.terminals[type_])
            except IndexError:
                _, _, traceback = sys.exc_info()
                raise IndexError("The gp.generate function tried to add "
                                 "a terminal of type '%s', but there is "
                                 "none available." % (type_,)).with_traceback(traceback)
            if inspect.isclass(term):
                term = term()
            expr.append(term)
        else:
            try:
                # Might not be respected if there is a type without terminal args
                if height <= depth or (depth >= min_ and random.random() < pset.terminalRatio):
                    primitives_with_only_terminal_args = [p for p in pset.primitives[type_] if
                                                          all([arg in terminal_types for arg in p.args])]

                    if len(primitives_with_only_terminal_args) == 0:
                        prim = random.choice(pset.primitives[type_])
                    else:
                        prim = random.choice(primitives_with_only_terminal_args)
                else:
                    prim = random.choice(pset.primitives[type_])
            except IndexError:
                _, _, traceback = sys.exc_info()
                raise IndexError("The gp.generate function tried to add "
                                 "a primitive of type '%s', but there is "
                                 "none available." % (type_,)).with_traceback(traceback)
            expr.append(prim)
            for arg in reversed(prim.args):
                stack.append((depth + 1, arg))
    return expr

In [4]:
expr = generate_safe(pset, 3, 10, terminal_types, type_=None)
tree = gp.PrimitiveTree(expr)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax, pset=pset)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genGrow, pset=pset, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)

function = gp.compile(tree, pset)

print(tree)

series_concate(series_not(series_not(series_or(series_and(series_lt(ema(df, 113, 24), ema(df, 127, 7)), series_not(series_lt_float(rsi(df, 37, 6, 25), 0.08318798561540997))), series_gt_float(rsi(df, 138, 7, 14), 0.7233748011202429)))), series_or(series_or(series_gt_float(rsi(df, 200, 20, 8), 0.03851392612861848), series_lt_float(rsi(df, 40, 22, 16), 0.4682977024508034)), series_or(series_or(series_gt_float(rsi(df, 59, 24, 18), 0.9170823230850844), series_lt(ema(df, 58, 4), ema(df, 96, 15))), series_gt_float(rsi(df, 101, 12, 8), 0.08929135200990057))))


In [5]:
df = pd.read_csv('./btc-usd.csv')

In [6]:
function(df)

,0,1
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
9996,True,True
9997,True,True
9998,True,True
9999,True,True
